In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)
df_raw = pd.read_csv('/Users/uppalavenkatesh/Documents/WEO2.csv')
year_cols = [str(y) for y in range(2010, 2025)]
df = df_raw.melt(
    id_vars=["COUNTRY", "INDICATOR"],
    value_vars=year_cols,
    var_name="Year",
    value_name="Value"
)
df["Year"] = df["Year"].astype(int)
countries = [
    "United States", "Germany", "Brazil", "India", 
    "South Africa", "United Kingdom", "Japan"
]

indicators = {
    "Gross domestic product (GDP), Constant prices, Percent change": "GDP Growth",
    "All Items, Consumer price index (CPI), Period average, percent change": "Inflation",
    "Unemployment rate": "Unemployment"
}

df = df[df["COUNTRY"].isin(countries) & df["INDICATOR"].isin(indicators.keys())]
df["INDICATOR"] = df["INDICATOR"].replace(indicators)

# --- Drop Missing Values ---
df.dropna(subset=["Value"], inplace=True)

# --- Normalize the Data (Z-Score) ---
df["Z_Score"] = df.groupby("INDICATOR")["Value"].transform(
    lambda x: (x - x.mean()) / x.std()
)

# --- Year-over-Year Change ---
df["YoY_Change"] = df.groupby(["COUNTRY", "INDICATOR"])["Value"].diff()

# --- Summary Table: Average per Country per Indicator ---
summary_table = df.groupby(["COUNTRY", "INDICATOR"])["Value"].mean().reset_index()
summary_pivot = summary_table.pivot(index="COUNTRY", columns="INDICATOR", values="Value")

# --- Generate Insight Summary ---
def generate_insight(summary_df):
    insights = []
    for country in summary_df.index:
        gdp = summary_df.loc[country, "GDP Growth"]
        inflation = summary_df.loc[country, "Inflation"]
        unemployment = summary_df.loc[country, "Unemployment"]
        insights.append(
            f"{country} had an average GDP growth of {gdp:.2f}%, "
            f"inflation of {inflation:.2f}%, and unemployment of {unemployment:.2f}% "
            f"from 2010 to 2024."
        )
    return "\n".join(insights)

insight_text = generate_insight(summary_pivot)

# --- Save GDP Growth Plot ---
gdp_df = df[df["INDICATOR"] == "GDP Growth"]
plt.figure(figsize=(12, 6))
sns.lineplot(data=gdp_df, x="Year", y="Value", hue="COUNTRY", marker="o")
plt.title("GDP Growth Over Time by Country")
plt.ylabel("GDP Growth (% Change)")
plt.xlabel("Year")
plt.grid(True)
plt.legend(title="Country")
plt.tight_layout()
gdp_plot_path = os.path.join(plots_dir, "gdp_growth.png")
plt.savefig(gdp_plot_path)
plt.close()

# --- Save Inflation & Unemployment Bar Chart ---
latest_year = df["Year"].max()
subset_df = df[(df["Year"] == latest_year) & df["INDICATOR"].isin(["Inflation", "Unemployment"])]
plt.figure(figsize=(12, 6))
sns.barplot(data=subset_df, x="COUNTRY", y="Value", hue="INDICATOR")
plt.title(f"Inflation & Unemployment Rates by Country ({latest_year})")
plt.ylabel("Rate (%)")
plt.xlabel("Country")
plt.xticks(rotation=45)
plt.tight_layout()
bar_plot_path = os.path.join(plots_dir, "inflation_unemployment.png")
plt.savefig(bar_plot_path)
plt.close()

# --- Heatmaps for Each Country ---
corr_data = df.pivot_table(
    index=["Year", "COUNTRY"],
    columns="INDICATOR",
    values="Value"
).dropna()

heatmap_paths = {}
for country in countries:
    country_data = corr_data.loc[(slice(None), country), :]
    if not country_data.empty:
        corr_matrix = country_data.corr()
        plt.figure(figsize=(6, 4))
        sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
        plt.title(f"Correlation Matrix: {country}")
        plt.tight_layout()
        heatmap_path = os.path.join(plots_dir, f"heatmap_{country.replace(' ', '_')}.png")
        plt.savefig(heatmap_path)
        plt.close()
        heatmap_paths[country] = heatmap_path

# --- Export to Excel with Images ---
with pd.ExcelWriter("IMF_Economic_Summary.xlsx", engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name="Full_Data", index=False)
    summary_table.to_excel(writer, sheet_name="Summary_Averages", index=False)
    summary_pivot.to_excel(writer, sheet_name="Summary_Pivot")

    # Write insights
    workbook = writer.book
    insight_ws = workbook.add_worksheet("Insights")
    writer.sheets["Insights"] = insight_ws
    for i, line in enumerate(insight_text.split('\n')):
        insight_ws.write(i, 0, line)

    # Insert main charts
    charts_ws = workbook.add_worksheet("Charts")
    writer.sheets["Charts"] = charts_ws
    charts_ws.insert_image("B2", gdp_plot_path)
    charts_ws.insert_image("B25", bar_plot_path)

    # Insert heatmaps
    heatmap_ws = workbook.add_worksheet("Heatmaps")
    writer.sheets["Heatmaps"] = heatmap_ws
    row = 1
    for country, path in heatmap_paths.items():
        heatmap_ws.write(row - 1, 0, country)
        heatmap_ws.insert_image(row, 1, path)
        row += 20

print("✅ All data, charts, and summaries exported to 'IMF_Economic_Summary.xlsx'")


✅ All data, charts, and summaries exported to 'IMF_Economic_Summary.xlsx'
